In [14]:
# importing modules
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar

In [15]:

rheas_district_corrected=pd.read_csv('data/rheas/rheas_preharvest_district_corrected.csv')
preharvest_district=pd.read_csv('data/preharvest/zambia_preharvest_district_data_clean.csv')

In [16]:
# aggregate data

preharvest=preharvest_district.groupby(['district'],as_index=False).agg({'yield':'mean'})
rheas_district_pre_corrected=rheas_district_corrected.groupby(['district_b'],as_index=False).agg({'yield_corrected':'mean'})
rheas_district=rheas_district_corrected.groupby(['district_b'],as_index=False).agg({'yield':'mean'})
preharvest


district     yield
0      chadiza  2.168820
1        chama  2.076612
2      chavuma  2.307771
3     chibombo  1.919281
4      chienge  2.216786
..         ...       ...
75    siavonga  0.885902
76  sinazongwe  0.897332
77     solwezi  1.772950
78       total  1.984708
79     zambezi  1.855669

[80 rows x 2 columns]

In [3]:
# Reading the data
# read files
rheas_chirps= pd.read_csv('data/rheas/zmb_n_25_chirps__newmask_1999_2023_maize.csv')
# # rheas_chirps['season'] = rheas_chirps['season'].astype(str)
# rheas_chirps['season2']=rheas_chirps['season'].str.extract(r'(\d{1}$)')
# rheas_chirps[rheas_chirps['yea']=='2']
len(rheas_chirps.cname.unique())

116

In [4]:
# convert columns to lower and cname names
rheas_chirps.columns=rheas_chirps.columns.str.lower()
rheas_chirps['cname']=rheas_chirps['cname'].str.lower()
rheas_chirps['cname']=rheas_chirps['cname'].str.strip()
rheas_chirps['cname']=rheas_chirps['cname'].str.replace('-', '', regex=True)

In [5]:
# turn to string
# rheas_chirps['planting']=rheas_chirps['planting'].to_string()

In [6]:
# convert dates and calculate yield
rheas_chirps['planting']=pd.to_datetime(rheas_chirps['planting'], format="mixed")
rheas_chirps['year'] = rheas_chirps['planting'].dt.year
rheas_chirps['yield']=rheas_chirps['gwad']/1000
rheas_chirps['month']=rheas_chirps['planting'].dt.month
rheas_chirps['month'].unique()

array([ 1, 10, 11, 12])

In [7]:
#season [10,11,12, 1,2,3,4,5,6]
# #  part1=[1,2, 3, 4,5,6], part1 = [10, 11, 12], season part1=[10, 11, 12]
# rheas_chirps_part1=rheas_chirps

# part1 = ['none', 'none', 'none','none','none','none','none','none','none','yes','yes','yes']
# month_to_part1=dict(zip(range(1,13), part1))
# rheas_chirps_part1['season']= rheas_chirps_part1['planting'].dt.month.map(month_to_part1)
# rheas_chirps_part1=rheas_chirps
# part1 = ['none', 'none', 'none','none','none','none','none','none','none','yes','yes','yes']
# month_to_part1=dict(zip(range(1,13), part1))
# rheas_chirps_part1['season']= rheas_chirps_part1['planting'].dt.month.map(month_to_part1)
# rheas_chirps_part1

rheas_chirps_part2=rheas_chirps
part = ['part2', 'part2', 'part2', 'part2','part2', 'part2', 'none', 'none', 'none', 'part1', 'part1', 'part1']
month_to_part=dict(zip(range(1,13), part))
rheas_chirps['season2']= rheas_chirps['planting'].dt.month.map(month_to_part)
# rheas_chirps_part1[rheas_chirps_part1['season']=='no']
rheas_chirps


id  gid       cname  season   ccode  ensemble   planting   
0       1083083   94     nkeyema  1998 1  110011        29 1999-01-01  \
1       1083087   94     nkeyema  1998 1  110011        33 1999-01-01   
2       1056596   83  mushindano  1998 1  108011        15 1999-01-01   
3       1085429   95       nsama  1998 1  107009        15 1999-01-01   
4        929310   30      kalabo  1998 1  110001        41 1999-01-01   
...         ...  ...         ...     ...     ...       ...        ...   
295774  1289442   63      mambwe  2022 1  103005         8 2023-01-30   
295775  1229437   38      kasama  2022 1  107003         3 2023-01-30   
295776  1229441   38      kasama  2022 1  107003         7 2023-01-30   
295777  1289479   63      mambwe  2022 1  103005        45 2023-01-30   
295778  1239071   42    kawambwa  2022 1  104004        37 2023-01-30   

           harvest   wsgd  gwad  year  yield  month season2  
0       1999-05-18  0.000  3716  1999  3.716      1   part2  
1       1999-05-18  0.000  3716  1999  3.716      1   part2  
2       1999-05-18  0.000  3716  1999  3.716      1   part2  
3       1999-05-18  0.000  3716  1999  3.716      1   part2  
4       1999-05-18  0.000  3716  1999  3.716      1   part2  
...            ...    ...   ...   ...    ...    ...     ...  
295774  2023-06-04  0.000  4090  2023  4.090      1   part2  
295775  2023-06-07  0.182  4447  2023  4.447      1   part2  
295776  2023-06-06  0.000  4161  2023  4.161      1   part2  
295777  2023-06-04  0.000  4230  2023  4.230      1   part2  
295778  2023-06-06  0.495  3358  2023  3.358      1   part2  

[295779 rows x 14 columns]

In [8]:
# Extract data only for the seasons
rheas_chirps_part1= rheas_chirps.loc[rheas_chirps['season2']=='part1']
rheas_chirps_part2= rheas_chirps.loc[rheas_chirps_part2['season2']=='part2']
# rheas_chirps_part2.rheas_chirps_part2.loc[rheas_chirps_part2['year']==2022]


In [9]:

# extract data for 2022 and 2023 only
rheas_chirps_2022_2=rheas_chirps_part2.loc[rheas_chirps_part2['year']==2021].reset_index()
rheas_chirps_2023_1=rheas_chirps_part1.loc[rheas_chirps_part1['year']==2022].reset_index()
rheas_chirps_2022=pd.concat([rheas_chirps_2022_2,rheas_chirps_2023_1],axis=0)
rheas_chirps_2023_1


index       id  gid         cname  season   ccode  ensemble   planting   
0     284179  1342192   85  mwansabombwe  2022 1  104008         8 2022-10-15  \
1     284180  1332586   81        mumbwa  2022 1  101009         2 2022-10-15   
2     284181  1315797   74         mpika  2022 1  106005        13 2022-10-15   
3     284182  1373432   98       petauke  2022 1  103007        48 2022-10-15   
4     284183  1327821   79      mufumbwe  2022 1  108006        37 2022-10-15   
...      ...      ...  ...           ...     ...     ...       ...        ...   
5773  289952  1349391   88       nakonde  2022 1  106006         7 2022-12-28   
5774  289953  1349425   88       nakonde  2022 1  106006        41 2022-12-28   
5775  289954  1349433   88       nakonde  2022 1  106006        49 2022-12-29   
5776  289955  1349401   88       nakonde  2022 1  106006        17 2022-12-30   
5777  289956  1349406   88       nakonde  2022 1  106006        22 2022-12-31   

         harvest  wsgd  gwad  year  yield  month season2  
0     2023-02-11   0.0  3805  2022  3.805     10   part1  
1     2023-02-11   0.0  3805  2022  3.805     10   part1  
2     2023-02-11   0.0  3805  2022  3.805     10   part1  
3     2023-02-11   0.0  3805  2022  3.805     10   part1  
4     2023-02-11   0.0  3805  2022  3.805     10   part1  
...          ...   ...   ...   ...    ...    ...     ...  
5773  2023-05-03   0.0  3230  2022  3.230     12   part1  
5774  2023-05-05   0.0  3315  2022  3.315     12   part1  
5775  2023-05-04   0.0  3092  2022  3.092     12   part1  
5776  2023-05-05   0.0  3071  2022  3.071     12   part1  
5777  2023-05-08   0.0  3218  2022  3.218     12   part1  

[5778 rows x 15 columns]

In [10]:
# # aggregate 
rheas_chirps_2022=rheas_chirps_2022.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_chirps_2022

cname    yield
0    chadiza  3.91975
1      chama  3.96394
2    chasefu  3.96293
3    chavuma  3.97110
4     chembe  3.97782
..       ...      ...
111    sioma  4.00198
112  solwezi  3.96886
113    vubwi  4.04518
114  zambezi  3.97710
115    zimba  3.98420

[116 rows x 2 columns]

In [11]:
# Read shapefiles and rename columns
country_shapefile = gpd.read_file("data/shp/gadm36_ZMB_0.shp")
district_shp = gpd.read_file("data/shp/zmb_district_cleaned.shp")
province_shp = gpd.read_file("data/shp/zmb_province_cleaned.shp")


# # Rename kenya_shp 
district_shp=district_shp.rename(columns={'district_b':'cname'})
# district_shp['cname']=district_shp['NAME_2']
# len(district_shp.cname.unique())
district_shp['cname']=district_shp['cname'].str.lower()
district_shp['cname']=district_shp['cname'].str.replace('-', '', regex=True)
district_shp['cname']=district_shp['cname'].str.strip()
len(district_shp.cname.unique())

116

In [12]:
# Merge data into shapefiles
tamsat_2022 = pd.merge(district_shp, rheas_chirps_2022, on="cname")

In [13]:
geo_source_tamsat_2022 = GeoJSONDataSource(geojson=tamsat_2022.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

In [14]:
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p1 = figure(title='Unimodal_Long Rains Season Yield (mt/ha) - 2022',
           outer_width=350, outer_height=300)

p1.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_source_tamsat_2022,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District: ', '@cname'), ('Yield', '@yield')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [15]:
# color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
# p2 = figure(title='Bimodal_MAM Season Yield (mt/ha) - 2022',
#            outer_width=350, outer_height=300)

# p2.multi_line('xs','ys', source=country_geo_source,
#             line_color='black', line_width=0.5)

# p2.patches('xs','ys', source=geo_source_mam_2022,
#           fill_color={'field': 'yield', 'transform': color_mapper},
#           fill_alpha=0.7, line_color='black', line_width=0.5)

# hover = HoverTool(tooltips=[('District: ', '@cname'), ('Yield', '@yield')])
# p2.add_tools(hover)

# color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
#                      border_line_color=None, location=(0,0), orientation='vertical')
# p2.add_layout(color_bar, 'right')

In [16]:
# # Bimodal Season 2022
# color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
# p3 = figure(title=' Bimodal_ MAM Season  Yield(mt/Ha) - 2023',
#            outer_width=350, outer_height=300)

# p3.multi_line('xs','ys', source=country_geo_source,
#             line_color='black', line_width=0.5)

# p3.patches('xs','ys', source=geo_source_mam_2023,
#           fill_color={'field': 'yield', 'transform': color_mapper1},
#           fill_alpha=0.7, line_color='black', line_width=0.5)

# hover = HoverTool(tooltips=[('District', '@cname'), ('Yield', '@yield')])
# p3.add_tools(hover)

# color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
#                      border_line_color=None, location=(0,0), orientation='vertical')
# p3.add_layout(color_bar, 'right')

In [17]:
# # Bimodal Season 2022
# color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
# p4 = figure(title='Bimodal_OND Season  Yield(mt/ha) - 2022',
#            outer_width=350, outer_height=300)

# p4.multi_line('xs','ys', source=country_geo_source,
#             line_color='black', line_width=0.5)

# p4.patches('xs','ys', source=geo_source_ond_2022,
#           fill_color={'field': 'yield', 'transform': color_mapper1},
#           fill_alpha=0.7, line_color='black', line_width=0.5)

# hover = HoverTool(tooltips=[('District', '@cname'), ('Yield', '@yield')])
# p4.add_tools(hover)

# color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
#                      border_line_color=None, location=(0,0), orientation='vertical')
# p4.add_layout(color_bar, 'right')

In [18]:

# grid=gridplot([ p2, p3, p4, p1],ncols=2, width=650, height=600)
# grid = gridplot([[p2, p3], [None, p4]], width=650, height=600)
grid = gridplot([p1,], ncols=1, width=650, height=600)
show(grid)

In [19]:
from bokeh.io import save
# # # specify the output file name and location
save(grid, filename='../zambia.html', title='Maize Yield Maps')

C:\Users\kasit\AppData\Local\Temp\ipykernel_24744\1141731985.py:3: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(grid, filename='../zambia.html', title='Maize Yield Maps')


'd:\\RONO\\rheas\\maize_crop_modeling\\crop-modeling\\zambia.html'